In [1]:
import importlib
import utils
importlib.reload(utils)
import utils
from utils import *

/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
xtrain_eeg1, xtrain_eeg2, xtrain_emg, ytrain, xtest_eeg1, xtest_eeg2, xtest_emg = load_data()

Loading xtrain...
Shapes: (64800, 512) (64800, 512) (64800, 512)
Loading ytrain...
Shape: (64800, 1)
Loading xtest...
Shapes: (43200, 512) (43200, 512) (43200, 512)


In [3]:
# CRF Model Preprocessing
eeg1_ = process_EEG(xtrain_eeg1)
eeg2_ = process_EEG(xtrain_eeg2)
emg_ = process_EMG(xtrain_emg)
xtrain_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)

eeg1_ = process_EEG(xtest_eeg1)
eeg2_ = process_EEG(xtest_eeg2)
emg_ = process_EMG(xtest_emg)
xtest_ = np.concatenate((eeg1_, eeg2_, emg_), axis=1)


/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:693: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (p, _, _, _) = numpy.linalg.lstsq(x, L)
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:243: RuntimeWarning: divide by zero encountered in power
  np.sum((np.power(np.abs(sig),(-0.3)) > 20), axis=1),
/Users/gabrielarpino/Documents/ETH_Semester_1/Advanced_Machine_Learning/sdaml/task4/gabriel/utils.py:693: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the ol

# RNN Implementation

In [ ]:
# LSTM
print("Running LSTM...")
seed=2
kfold = KFold(n_splits=3, shuffle=False, random_state=seed)
EPOCHS=20
timesteps = 21600
classes = np.array([1,2,3])
num_classes=len(classes)
ytrain_classes = np.reshape(ytrain.values, (ytrain.shape[0],))
weights = compute_class_weight("balanced", list(classes), list(ytrain_classes))
cw = dict(zip(np.array([0,1,2]), weights))
lstm_predictions = []
data_dim = xtrain_.shape[1]
batch_size=32
res = []

X_train = xtrain_
y_train = ytrain.values

# X_test_scaled = preprocessing.StandardScaler().fit_transform(X_test)
# X_test_scaled_lstm = np.reshape(X_test_scaled, (2 * timesteps, 1, data_dim))

for train, valid in kfold.split(X_train):
    print("  start fold")
    X_train_fold = X_train[train]
    y_train_fold = y_train[train]
    
    X_valid_fold = X_train[valid]
    y_valid_fold = y_train[valid]

    X_train_fold_scaled = preprocessing.StandardScaler().fit_transform(X_train_fold)
    X_valid_fold_scaled = preprocessing.StandardScaler().fit_transform(X_valid_fold)
    
    X_train_fold_scaled_lstm = np.reshape(X_train_fold_scaled, (2 * timesteps, 1, data_dim))
    y_train_fold_lstm = from_label_to_vec(y_train_fold)
    
    X_valid_fold_scaled_lstm= np.reshape(X_valid_fold_scaled, (timesteps, 1, data_dim))
    y_valid_fold_lstm = from_label_to_vec(y_valid_fold)


    model = Sequential()
    model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, stateful=True, batch_input_shape=(batch_size, 1, data_dim)))
    model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, stateful=True))
    model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2, stateful=True))
    model.add(Dense(num_classes, activation='softmax'))
    

    model.compile(loss=weighted_categorical_crossentropy(weights),
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    model.fit(X_train_fold_scaled_lstm, y_train_fold_lstm,
              class_weight=cw,
              verbose=1,
              batch_size=batch_size, epochs=EPOCHS, shuffle=False,
              validation_data=(X_valid_fold_scaled_lstm, y_valid_fold_lstm))
    
    
    
    y_pred_lstm = model.predict(X_valid_fold_scaled_lstm)
    y_pred = np.reshape(from_vec_to_labels(y_pred_lstm), (y_valid_fold.shape[0], y_valid_fold.shape[1]))
    resy = sklearn.metrics.balanced_accuracy_score(y_valid_fold, y_pred)
    print("RES:", resy)
 
    res.append(resy)
    
# y_pred_lstm = (lstm_predictions[0] + lstm_predictions[1] + lstm_predictions[2])/3.0
# labels = []
# for p in y_pred_lstm:
#     labels.append(np.argmax(p)+1)
#     y_pred_lstm = np.array(labels)   
    
# y_pred_filtered_lstm = medfilt(y_pred_lstm, FILTER_WINDOW)

# print(f"LSTM unique predictions {np.unique(y_pred_filtered_lstm)}")

# Grid search/

In [4]:
epochs = [10, 13, 15]
batch_size = [32]
dropout = [0.05, 0.15, 0.35]
recurrent_dropout = [0.05, 0.15, 0.35]
optimizer = ['adam']
units = [32]
total_res = []
for e, b, d, r, o, u in itertools.product(epochs, batch_size, dropout, recurrent_dropout, optimizer, units):
    res = losocv_RNN_prepro(xtrain_, ytrain, epochs=e, batch_size=b, 
                            dropout=d, recurrent_dropout=r, optimizer=o, units=u, verbose=0)
    total_res.append(res)
    print("e, b, d, r, o, u :", (e, b, d, r, o, u), ", RES:", (np.mean(res), np.std(res)), res)

Running LSTM...
Instructions for updating:
Colocations handled automatically by placer.


20-Dec-19 18:38:53 | WARNING | From /anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


20-Dec-19 18:38:53 | WARNING | From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


20-Dec-19 18:38:57 | WARNING | From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


e, b, d, r, o, u : (10, 32, 0.05, 0.05, 'adam', 32) , RES: (0.9189432025350103, 0.022964623750294514) [0.8896540642267724, 0.9457397550194161, 0.9214357883588425]
Running LSTM...
e, b, d, r, o, u : (10, 32, 0.05, 0.15, 'adam', 32) , RES: (0.9233319258321059, 0.03137645565417306) [0.8794939435871444, 0.9512004075752419, 0.9393014263339312]
Running LSTM...
e, b, d, r, o, u : (10, 32, 0.05, 0.35, 'adam', 32) , RES: (0.9283506237858109, 0.023380314528335132) [0.8958199828821866, 0.9497420258762043, 0.939489862599042]
Running LSTM...
e, b, d, r, o, u : (10, 32, 0.15, 0.05, 'adam', 32) , RES: (0.9210523502875296, 0.019662840543689716) [0.8932449254305999, 0.9349919721786089, 0.9349201532533802]
Running LSTM...
e, b, d, r, o, u : (10, 32, 0.15, 0.15, 'adam', 32) , RES: (0.9294595894943777, 0.024780050529682386) [0.8954764214476283, 0.9538629596392089, 0.9390393873962956]
Running LSTM...
e, b, d, r, o, u : (10, 32, 0.15, 0.35, 'adam', 32) , RES: (0.9203463500983201, 0.021635884917603335) [0.88

KeyboardInterrupt: 

In [ ]:
# Make a prediction with this best LSTM and send to nico.
# Best on this grid search: e, b, d, r, o, u : (13, 32, 0.1, 0.2, 'adam', 32) , RES: (0.9280961046150492, 0.021295602848577753) [0.899192108262748, 0.9498743089837762, 0.9352218965986238]
pred = RNN_pred_prepro(xtrain_, ytrain, xtest_, epochs=13, batch_size=32, dropout=0.1, recurrent_dropout=0.2, optimizer='adam', units=32, verbose=1)
make_submission('RNN_13_32_0.1_0.2_a_32.csv', pred)